# Setup

- Kør jsk.Isp Projektet.
    - _Du kan også bruge din egen Identity server_
- Kør Koden Nedenunder.

In [7]:
#r "nuget: System.Net.Http"
#r "nuget: System.Net.Http.Json"
#r "nuget: System.Text.Json"
#r "nuget: Flurl"
#r "nuget: Flurl.Http"
#r "nuget: IdentityModel"

using Microsoft.DotNet.Interactive.Formatting;
using System.Net.Http;

// Extension methods til brug med 0auth
using IdentityModel.Client;

// Flurl (Fluent url)
using Flurl;
using Flurl.Util;
using Flurl.Http;

// Konfiguration af client 
string _ClientId = "deviceclient"; // ClientId som clienten har
string _ClientSecret = "DeviceSecret"; // Client Secret som clienten har
string _Endpoint = "https://localhost:5001"; // Url Addressen til identity serveren

// Api
string _ApiEndpoint = "https://localhost:7216";

// Global Accesstoken variabel
string _AccessToken = string.Empty;

// Http client som bliver brugt hele tiden (Best practice)
var client = new HttpClient();

$"""
Endpoint: {_Endpoint}
Client Credentials: 
    Id: {_ClientId} 
    Secret: {_ClientSecret}
"""

Installed Packages Flurl, 3.0.7 Flurl.Http, 3.2.4 IdentityModel, 6.0.0 System.Net.Http, 4.3.4 System.Net.Http.Json, 7.0.0 System.Text.Json, 7.0.1

Endpoint: https://localhost:5001
Client Credentials: 
    Id: deviceclient 
    Secret: DeviceSecret

# 1. Send Authorization Request

Sender en Authorisation Request og modtager en lille kode som brugeren skal skrive ind for at logge ind på enheden

In [ ]:
// Send Request
var Device_Request  = await client.RequestDeviceAuthorizationAsync(new DeviceAuthorizationRequest
{
    Address = $"{_Endpoint}/connect/deviceauthorization",
    ClientId = _ClientId,
    ClientSecret = _ClientSecret,
    Scope = "openid profile api"
});

// Check for errors
if(!Device_Request.IsError)
{
    $"Device Code: {Device_Request.DeviceCode}".Display();

    $"""
    Login
    {_Endpoint}/device?userCode={Device_Request.UserCode} eller gå til {_Endpoint}/device og indtast kode
    Kode: {Device_Request.UserCode}
    """.Display();
}
else
{
    Device_Request.Error.Display();
}

# 2. Poll for Access token

Imens brugeren logger ind vil enheden begynde at polle Authorisation serveren og vil få tomme svar indtil brugren er logget ind.

> Start koden når du har kørt den øverste.

In [ ]:
var Request = new DeviceTokenRequest() {
    Address = $"{_Endpoint}/connect/token",
    ClientId = _ClientId,
    ClientSecret = _ClientSecret,
    DeviceCode = Device_Request.DeviceCode
};

Console.Write("Waiting for user to consent.");

// Tom Response til brug i løkke
TokenResponse Response = new TokenResponse() {};

// Check hvis brugeren er logget ind
do
{

    Response = await client.RequestDeviceTokenAsync(Request);

    if(Response.IsError)
    {
        Console.Write(".");
        await Task.Delay(5000);
    }
    else
    {
        Console.WriteLine("\nAccess Token Recieved!!!");
    }


} 
while(Response.IsError);

// Gem AccessToken til senere
_AccessToken = Response.AccessToken;

$"""
Access token: {Response.AccessToken}
Type: {Response.TokenType}
Expires: {DateTime.UnixEpoch.AddDays(Response.ExpiresIn).ToString()}
""".Display();

# 3. Hent Data Fra Api
Hent Alle Todos Fra Apien med den gyldige Access Token.

> Bruger Flurl.

In [11]:
// Todo klasse til opbevaring af data
public class Todo {
    public int Id { get; set; }
    public string Title { get; set; }
    public string Description { get; set; }
    public DateTime CreatedDate { get; set;}
    public int UserId { get; set; }
}

// Bruger den globale client til At lave en FlurlCLient
FlurlClient flurlClient = new(client);

// Starter en query til Api
var query = flurlClient.Request(_ApiEndpoint);

// tilføjer berarer token til query, hvis der er en.
if(_ClientId != string.Empty)
{
    query = query.WithOAuthBearerToken(_AccessToken);
}

// Retter Uri til At hente alle Todos. ({URL}/api/todos/all)
query = query.AppendPathSegments("api","todos","all");

var result = await query.GetJsonAsync<List<Todo>>();

result.Display();

index,Id,Title,Description,CreatedDate,UserId
0,1,Test1,Test1,2023-02-03 12:43:29Z,1
1,2,Test2,Test2,2023-02-03 12:43:29Z,2
2,3,Test3,Tes3t,2023-02-03 12:43:29Z,2
3,4,Test4,Test4,2023-02-03 12:43:29Z,1
